# Project Details
Commercial banks receive a lot of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. **Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!)**. Luckily, this task can be automated with the power of **machine learning** and pretty much every commercial bank does so nowadays. 
<p><img src="https://assets.datacamp.com/production/project_558/img/credit_card.jpg" alt="Credit card being held in hand"></p>

### Objectives
* Building an automatic credit card approval predictor using machine learning techniques, just like the real banks do.
* We will build a machine learning model that can predict **if an individual's application for a credit card will be accepted or not**.

### Data Source
> We'll use the <a href="http://archive.ics.uci.edu/ml/datasets/credit+approval">Credit Card Approval dataset</a> from the UCI Machine Learning Repository. The structure of this notebook is as follows:

# Preparing & Processing Data

In [318]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn

In [319]:
cc_approvals = pd.read_csv('cc_approvals.data', header=None)

cc_approvals.info()
cc_approvals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


#### Attribute Information:

    A1:	b, a.
    A2:	continuous.
    A3:	continuous.
    A4:	u, y, l, t.
    A5:	g, p, gg.
    A6:	c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
    A7:	v, h, bb, j, n, z, dd, ff, o.
    A8:	continuous.
    A9:	t, f.
    A10:	t, f.
    A11:	continuous.
    A12:	t, f.
    A13:	g, p, s.
    A14:	continuous.
    A15:	continuous.
    A16: +,-         (class attribute)

The output may appear a bit confusing at its first sight, but let's try to figure out the most important features of a credit card application. The features of this dataset have been anonymized to protect the privacy, but this blog gives us a pretty good overview of the probable features. The probable features in a typical credit card application are **Gender, Age, Debt, Married, BankCustomer, EducationLevel, Ethnicity, YearsEmployed, PriorDefault, Employed, CreditScore, DriversLicense, Citizen, ZipCode, Income and finally the ApprovalStatus**. This gives us a pretty good starting point, and we can map these features with respect to the columns in the output.

In [320]:
cc_approvals.describe()

,2,7,10,14
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [321]:
cc_approvals.tail(20).head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
670,b,47.17,5.835,u,g,w,v,5.50,f,f,0,f,g,00465,150,-
671,b,25.83,12.835,u,g,cc,v,0.50,f,f,0,f,g,00000,2,-
672,a,50.25,0.835,u,g,aa,v,0.50,f,f,0,t,g,00240,117,-
673,?,29.50,2.000,y,p,e,h,2.00,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.21,f,f,0,f,g,00260,246,-


### Replacing missing data
The missing values in the dataset are labeled with '?'

In [322]:
cc_approvals = cc_approvals.replace('?', np.nan)

In [323]:
cc_approvals.tail(17).head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,NaN,29.50,2.0,y,p,e,h,2.00,f,f,0,f,g,00256,17,-
674,a,37.33,2.5,u,g,i,h,0.21,f,f,0,f,g,00260,246,-


In [266]:
cc_approvals.isna().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

In [267]:
cc_approvals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    object 
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      677 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


An important question that gets raised here is why are we giving so much importance to missing values? Can't they be just ignored? **Ignoring missing values can affect the performance of a machine learning model heavily**. While ignoring the missing values our machine learning model may miss out on information about the dataset that may be useful for its training. Then, there are many models which cannot handle missing values implicitly such as LDA.

So, to avoid this problem, we are going to impute the missing values with a strategy called **mean imputation**.

In [268]:
# cc_approvals.fillna(cc_approvals.mean(), inplace=True)

In [269]:
cc_approvals.tail(17)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,NaN,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


We have successfully taken care of the missing values present in the numeric columns. There are still some missing values to be imputed for columns 0, 1, 3, 4, 5, 6 and 13. All of these columns contain non-numeric data and this why the mean imputation strategy would not work here. This needs a different treatment.

We are going to impute these missing values with **the most frequent values** as present in the respective columns. This is good practice when it comes **to imputing missing values for categorical data** in general.

In [270]:
cc_approvals.isna().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

In [271]:
cc_approvals.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')

In [272]:
cc_approvals[0].dtypes

dtype('O')

In [273]:
c = cc_approvals[0].value_counts()
c

b    468
a    210
Name: 0, dtype: int64

In [274]:
c.index[0]

'b'

In [275]:
# Iterate over each column of cc_apps
for col in cc_approvals.columns:
    # Check if the column is of object type
    if cc_approvals[col].dtypes == 'object':
        # Impute with the most frequent value
        cc_approvals = cc_approvals.fillna(cc_approvals[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify
cc_approvals.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64

# Model Building

### Convert the non-numeric data into numeric.

In [277]:
cc_approvals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [278]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Iterate over all the values of each column and extract their dtypes
for col in cc_approvals.columns:
    # Compare if the dtype is object
    if cc_approvals[col].dtypes=='O':
    # Use LabelEncoder to do the numeric transformation
        cc_approvals[col]=le.fit_transform(cc_approvals[col])

In [279]:
cc_approvals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    int32  
 1   1       690 non-null    int32  
 2   2       690 non-null    float64
 3   3       690 non-null    int32  
 4   4       690 non-null    int32  
 5   5       690 non-null    int32  
 6   6       690 non-null    int32  
 7   7       690 non-null    float64
 8   8       690 non-null    int32  
 9   9       690 non-null    int32  
 10  10      690 non-null    int64  
 11  11      690 non-null    int32  
 12  12      690 non-null    int32  
 13  13      690 non-null    int32  
 14  14      690 non-null    int64  
 15  15      690 non-null    int32  
dtypes: float64(2), int32(12), int64(2)
memory usage: 54.0 KB


### Splitting the dataset into train and test sets
Also, features like **DriversLicense and ZipCode** are not as important as the other features in the dataset for predicting credit card approvals. We should drop them to design our machine learning model with the best set of features. In Data Science literature, this is often referred to as `feature selection`.

In [280]:
from sklearn.model_selection import train_test_split
# Drop the features 11 and 13 and convert the DataFrame to a NumPy array
cc_approvals = cc_approvals.drop([11, 13], axis=1)

In [281]:
cc_approvals.head(2)

,0,1,2,3,4,5,6,7,8,9,10,12,14,15
0,1,156,0.00,2,1,13,8,1.25,1,1,1,0,0,0
1,0,328,4.46,2,1,11,4,3.04,1,1,6,0,560,0


In [282]:
cc_approvals = cc_approvals.to_numpy()

In [283]:
cc_approvals

array([[1.000e+00, 1.560e+02, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 3.280e+02, 4.460e+00, ..., 0.000e+00, 5.600e+02,
        0.000e+00],
       [0.000e+00, 8.900e+01, 5.000e-01, ..., 0.000e+00, 8.240e+02,
        0.000e+00],
       ...,
       [0.000e+00, 9.700e+01, 1.350e+01, ..., 0.000e+00, 1.000e+00,
        1.000e+00],
       [1.000e+00, 2.000e+01, 2.050e-01, ..., 0.000e+00, 7.500e+02,
        1.000e+00],
       [1.000e+00, 1.970e+02, 3.375e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00]])

In [298]:
X = cc_approvals[:, 0:13]
y = cc_approvals[:, 13]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


### Scaling

The data is now split into two separate sets - train and test sets respectively. We are only left with one final preprocessing step of scaling before we can fit a machine learning model to the data.

Now, let's try to understand what these scaled values mean in the real world. Let's use CreditScore as an example. The credit score of a person is their creditworthiness based on their credit history. The higher this number, the more financially trustworthy a person is considered to be. So, a CreditScore of 1 is the highest since we're rescaling all the values to the range of 0-1.

In [299]:
from sklearn.preprocessing import MinMaxScaler

# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

### Fitting a logistic regression model to the train set
Essentially, predicting if a credit card application will be approved or not is a **classification task**. According to UCI, our dataset contains more instances that correspond to "Denied" status than instances corresponding to "Approved" status. Specifically, out of 690 instances, there are 383 (55.5%) applications that got denied and 307 (44.5%) applications that got approved.

This gives us a benchmark. A good machine learning model should be able to accurately predict the status of the applications with respect to these statistics.

**Which model should we pick?** A question to ask is: are the features that affect the credit card approval decision process correlated with each other? Although we can measure correlation, that is outside the scope of this notebook, so we'll rely on our intuition that they indeed are correlated for now. Because of this correlation, we'll take advantage of the fact that **generalized linear models perform well in these cases**. Let's start our machine learning modeling with a Logistic Regression model (a generalized linear model).

In [300]:
from sklearn.linear_model import LogisticRegression

# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()

# Fit logreg to the train set
logreg.fit(rescaledX_train, y_train)

LogisticRegression()

### Making predictions and evaluating performance
But how well does our model perform?

We will now evaluate our model on the test set with respect to **classification accuracy**. But we will also take a look the model's **confusion matrix**. In the case of predicting credit card applications, it is equally important to see if our machine learning model is able to predict the approval status of the applications as denied that originally got denied. If our model is not performing well in this aspect, then it might end up approving the application that should have been approved. The confusion matrix helps us to view our model's performance from these aspects.

In [301]:
from sklearn.metrics import confusion_matrix

y_pred = logreg.predict(rescaledX_test)

print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test, y_test))

confusion_matrix(y_test, y_pred)

Accuracy of logistic regression classifier:  0.8421052631578947


array([[94,  9],
       [27, 98]], dtype=int64)

Our model was pretty good! It was able to yield an accuracy score of almost 84%.

For the confusion matrix, the first element of the of the first row of the confusion matrix denotes the `true negatives` meaning the number of negative instances (**denied applications**) predicted by the model correctly. And the last element of the second row of the confusion matrix denotes the `true positives` meaning the number of positive instances (**approved applications**) predicted by the model correctly.

### Grid searching and making the model perform better
Let's see if we can do better. We can perform a grid search of the model parameters to improve the model's ability to predict credit card approvals.

scikit-learn's implementation of logistic regression consists of different hyperparameters but we will grid search over the following two:

* tol
* max_iter

In [312]:
from sklearn.model_selection import GridSearchCV

# Define the grid of values for tol and max_iter
tol = [0.01, 0.001, 0.001]
max_iter = [100, 150, 200, 300]

# Create a dictionary where tol and max_iter are keys and the lists of their values are corresponding values
param_grid = dict(tol = tol, max_iter = max_iter)

In [313]:
param_grid

{'tol': [0.01, 0.001, 0.001], 'max_iter': [100, 150, 200, 300]}

#### GridSearchCV 
We will instantiate GridSearchCV() with our earlier logreg model with all the data we have. **Instead of passing train and test sets separately, we will supply X (scaled version) and y**. We will also instruct GridSearchCV() to perform a cross-validation of five folds.

In [316]:
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Use scaler to rescale X and assign it to rescaledX
rescaledX = scaler.fit_transform(X)

# Fit data to grid_model
grid_model_result = grid_model.fit(rescaledX, y)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.850725 using {'max_iter': 100, 'tol': 0.01}


# End
While building this credit card predictor, we tackled some of the most widely-known preprocessing steps such as `scaling, label encoding, and missing value imputation`. We finished with some machine learning to predict if a person's application for a credit card would get approved or not given some information about that person.

**Note**

This project is one of DataCamp Projects.

DONE ON: 24.09.2021